In [1]:
import sys
import pandas as pd 
sys.path.insert(0, '/home/bekah/gPhoton2')
from gPhoton.pipeline import execute_pipeline
from astropy.io import fits
from io import BytesIO
import numpy as np 
import os
import matplotlib.pyplot as plt
from gPhoton.coadd import zero_flag_and_edge
import pyarrow.parquet as parquet
from quickbin import bin2d

from photutils.background import Background2D, MedianBackground
from astropy.stats import SigmaClip

In [2]:
# make a list of fuv to make / look at 

og_aspect = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/metadata.parquet').to_pandas()
og_aspect = og_aspect.dropna(subset=['fuv_temp'])

In [3]:
og_aspect = og_aspect[og_aspect['legs'] == 0]

sampled_eclipses = og_aspect.sample(n=200, random_state=2)
sampled_eclipses

,eclipse,obstype,legs,ra_max,ra_min,dec_max,dec_min,fuv_temp
8261,13338,DIS,0,53.120694,53.091952,-29.389283,-29.414299,26.811481
971,2870,DIS,0,53.147050,53.119313,-27.857205,-27.880872,27.756960
5456,9405,GII,0,150.396398,150.373411,-6.591745,-6.615505,27.369032
13246,19188,GII,0,47.529317,47.491357,-53.324985,-53.347183,28.140380
3092,6307,NGS,0,273.377713,273.351977,9.065354,9.039640,29.535768
...,...,...,...,...,...,...,...,...
5634,9757,GII,0,122.818798,122.790868,25.218543,25.193259,31.382072
4301,7788,GII,0,2.483505,2.454958,25.940227,25.914979,29.628210
22304,30627,MIS,0,131.845922,131.817471,26.710279,26.684506,30.203828
18480,25782,MIS,0,125.298838,125.282610,8.016765,8.000432,29.571084


In [4]:
for index, eclipse in sampled_eclipses['eclipse'].items():
    print(f"running {eclipse}")
    try: 
        execute_pipeline(
                int(eclipse),
                "FUV",
                depth=None,
                # integer; None to deactivate (default None)
                threads=4,
                # where to both write output data and look for input data
                local_root="/media/bekah/BekahA/background_test/pics_and_photonlists",
                # auxiliary remote location for input data
                # remote_root="/mnt/s3",
                recreate=True,
                # list of floats; relevant only to lightcurve / photometry portion
                aperture_sizes=[12.8],
                # actually write image/movie products? otherwise hold in memory but
                # discard (possibly after performing photometry).
                write={"movie": False, "image": True},
                coregister_lightcurves=False,
                # photonpipe, moviemaker, None (default None)
                stop_after=None,
                photometry_only=False,
                # None, "gzip", "rice"
                compression="rice",
                # use array sparsification on movie frames?
                lil=False,
                # write movie frames as separate files
                burst=False,
                extended_photonlist=True,
                # aspect file, don't need to set unless need to use alt
                # file, 'aspect2.parquet'
            )
        
    except Exception as e: 
        print(e)


running 13338


/home/bekah/gPhoton2/gPhoton/pipeline.py:231: UserWarning: lil=False is deprected and will be removed in a future release. Defaulting to lil=True.
  warnings.warn("lil=False is deprected and will be removed in a future release. Defaulting to lil=True.")


starting timer
eclipse 13338 FUV  -- DIS; 0 leg(s)
Processing eclipse 13338
Offsetting FUV image for eclipse 13338 at 26.8114806821977 degrees.
Setting FUV offsets to x=-0.19512, y=1.0471003986134872
trange= ( 814498937.995 , 814499455.995 )                   
RA AVG: 53.10677801796305, DEC AVG: -29.401044353854793, ROLL AVG: 153.99720944406084
Loading raw6 file...                                        
829695 events
Unpacking raw6 data...                                      
stim_coef0, stim_coef1 = -442204.7380014617, 0.0005491652432270735
writing table to /media/bekah/BekahA/background_test/pics_and_photonlists/e13338/e13338-fd-b00.parquet
                                                            
Runtime statistics:
 runtime		=	2.28 sec. = (0.04 min.)
  processed	   =   827344 of 829695 events.
rate		=	364498.67 photons/sec.

masking photonlist
reading photonlist
filtering photonlist
calculating expt & adding edge points
quickbinning
masking binned data
making & saving new mask

KeyboardInterrupt: 

In [ ]:
# redo above without the .5 subtraction of flat field 